### Day 9

In [1]:
# Load input
with open("inputs/day9_input.dat","r") as f:
    data = f.read().splitlines()

for ix,row in enumerate(data):
    data[ix] = [int(x) for x in row]
data[0][0:5]

[7, 6, 7, 8, 9]

### Part 1:
- Values are heights
- Find all of the local minima
- Risk value of local minima is height +1
- What is the sume of all risk values for all of the low points?

Thoughts:
- Can probably test every point in reasonable time
- Otherwise could test every second point and do a small gradient descent-like method, keeping track of where we've been

In [2]:
class HeightMap(object):
    def __init__(self,height_map):
        self.map = height_map
        self.map_sz_x = len(height_map)
        self.map_sz_y = len(height_map[0])
    
    def check_if_low_point(self,x,y):
        val = self.map[x][y]
        # Check above
        if x>0:
            if self.map[x-1][y] <= val:
                return False
        # Check below
        if x < self.map_sz_x-1:
            if self.map[x+1][y] <= val:
                return False
        # Check left
        if y > 0:
            if self.map[x][y-1] <= val:
                return False
        # Check right
        if y < self.map_sz_y-1:
            if self.map[x][y+1] <= val:
                return False
        # Otherwise it's a min
        return True
    
    def find_low_points(self, verbose=False):
        low_points = []
        for x in range(self.map_sz_x):
            for y in range(self.map_sz_y):
                if self.check_if_low_point(x,y):
                    if verbose:
                        print("Low point at",x,y)
                    low_points.append([x,y])
        return low_points
        

    def sum_risk_values(self,verbose=False):
        low_points = self.find_low_points()
        risk_values = 0
        for x,y in low_points:
            risk_values += self.map[x][y]+1
        return risk_values

In [3]:
# Test input
test_input = [
    [2,1,9,9,9,4,3,2,1,0],
    [3,9,8,7,8,9,4,9,2,1],
    [9,8,5,6,7,8,9,8,9,2],
    [8,7,6,7,8,9,6,7,8,9],
    [9,8,9,9,9,6,5,6,7,8]
]
test_map = HeightMap(test_input)

test_map.sum_risk_values(True)

15

In [4]:
# Puzzle input
input_map = HeightMap(data)
input_map.sum_risk_values()

550

### Part 2:
- Basins are all points which lead to the same point when going downhill
- Height 9 doesnt count as part of a basin
- What is the multiple of the size of the 3 largest basins?

Thoughts:
- Loop through points that aren't 9, then move downhill and add 1 to a map once we reach the bottom.
- Then multiply the 3 largest together

In [5]:
class BasinMap(HeightMap):
    def __init__(self,height_map):
        super().__init__(height_map)
        basin_counts = []
        for x in range(self.map_sz_x):
            row = [0 for y in range(self.map_sz_y)]
            basin_counts.append(row)
        self.basin_counts = basin_counts
        
    def find_basin(self,x,y):
        val = self.map[x][y]
        neighbors = [val,9,9,9,9] # [here,top,bottom,left,right]
        # Check above
        if x>0:
            neighbors[1] = self.map[x-1][y]
        # Check below
        if x < self.map_sz_x-1:
            neighbors[2] = self.map[x+1][y]
        # Check left
        if y > 0:
            neighbors[3] = self.map[x][y-1]
        # Check right
        if y < self.map_sz_y-1:
            neighbors[4] = self.map[x][y+1]
        
        # Find the min and move there or add one here
        min_val = min(neighbors)
        best_dir = neighbors.index(min_val)
        if best_dir == 0:
            self.basin_counts[x][y] += 1
        elif best_dir == 1:
            self.find_basin(x-1,y)
        elif best_dir == 2:
            self.find_basin(x+1,y)
        elif best_dir == 3:
            self.find_basin(x,y-1)
        elif best_dir == 4:
            self.find_basin(x,y+1)
    
    def find_all_basins(self):
        for x in range(self.map_sz_x):
            for y in range(self.map_sz_y):
                if self.map[x][y] != 9:
                    self.find_basin(x,y)
        
        # Find the top 3 from the map
        top3 = [0,0,0]
        for x in range(self.map_sz_x):
            for y in range(self.map_sz_y):
                if self.basin_counts[x][y] > min(top3):
                    top3.remove(min(top3))
                    top3.append(self.basin_counts[x][y])
        return top3

        

In [6]:
# Test input
test_input = [
    [2,1,9,9,9,4,3,2,1,0],
    [3,9,8,7,8,9,4,9,2,1],
    [9,8,5,6,7,8,9,8,9,2],
    [8,7,6,7,8,9,6,7,8,9],
    [9,8,9,9,9,6,5,6,7,8]
]
test_map = BasinMap(test_input)

top3 = test_map.find_all_basins()
print("Multiple of top 3:",top3[0]*top3[1]*top3[2])
test_map.basin_counts

Multiple of top 3: 1134


[[0, 3, 0, 0, 0, 0, 0, 0, 0, 9],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 14, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 9, 0, 0, 0]]

In [7]:
# Puzzle input
basin_map = BasinMap(data)

top3 = basin_map.find_all_basins()
print("Multiple of top 3:",top3[0]*top3[1]*top3[2])

Multiple of top 3: 1100682
